In [33]:
import librosa
import matplotlib.pyplot as plt
import numpy as np

from IPython.display import Audio, display
from ipywidgets import FileUpload, Output, HBox, VBox, Layout

In [34]:
def handle_upload1(change):
    name = change['new'][0]['name']
    file_path = f"samples/{name}"
    with player1:
        player1.clear_output()
        print(file_path)
        display(Audio(file_path))

In [35]:
def handle_upload2(change):
    name = change['new'][0]['name']
    file_path = f"samples/{name}"
    with player2:
        player2.clear_output()
        print(file_path)
        display(Audio(file_path))

In [36]:
upload1 = FileUpload(multiple=False)
player1 = Output()
upload2 = FileUpload(multiple=False)
player2 = Output()

upload1.observe(handle_upload1, names='value')
upload2.observe(handle_upload2, names='value')

boxes = HBox([
    VBox([upload1, player1], layout= Layout(width='50%')),
    VBox([upload2, player2], layout= Layout(width='50%'))
])
display(boxes)

In [ ]:
# filename = librosa.example('nutcracker')
# y, sr = librosa.load(filename)


# # Plot the array
# plt.plot(y)

# # Set the title and labels of the plot
# plt.title("Line Plot of Array")
# plt.xlabel("Index")
# plt.ylabel("Value")

# # Show the plot
# plt.show()